In [65]:
from catboost import CatBoostClassifier, Pool
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

data = pd.read_csv('./user_data/feature.csv')

use_features = [col for col in data.columns if col not in ['sn','fault_time','Unnamed: 0','fault_time_ts','clean_event','event','label','Unnamed: 0.1']]
train = data[use_features]

print(train.columns)

train_data = train
train_labels = data['label']

train_pool = Pool(train_data,train_labels,cat_features=[0])

NUM_CLASSES = data['label'].nunique()
classes = np.unique(data['label'])
weights = compute_class_weight(class_weight='balanced', classes=classes, y=data['label'])
class_weights = dict(zip(classes, weights))
params = { 
            'task_type': 'CPU', 
            'bootstrap_type': 'Bernoulli',
            'learning_rate': 0.02, 
            'eval_metric': 'MultiClass', 
            'loss_function': 'MultiClass', 
            'classes_count': NUM_CLASSES, 
            'iterations': 500, 
            'random_seed': 2022, 
            'depth': 10, 
            'subsample': 0.8, 
            'leaf_estimation_iterations': 20,
            'reg_lambda': 0.5,
            'class_weights': class_weights,
            'early_stopping_rounds': 200 
        }
model = CatBoostClassifier(**params)

model.fit(train_pool,
       verbose=False,
       plot=True)

Index(['server_model', 'HDD', 'expired', 'designer', 'BP_HDD_Status', 'state',
       'ButtonMM', 'almost', 'Boot', 'hardware',
       ...
       'PCIE', 'IDE', 'present', 'EV', 'Sync', 'MB', 'Configuration',
       'Specific', 'OS_Boot', 'completed'],
      dtype='object', length=406)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [66]:
from catboost import Pool, cv

params = { 
            'task_type': 'CPU', 
            'bootstrap_type': 'Bernoulli',
            'learning_rate': 0.02, 
            'eval_metric': 'MultiClass', 
            'loss_function': 'MultiClass', 
            'classes_count': NUM_CLASSES, 
            'iterations': 500, 
            'random_seed': 2022, 
            'depth': 10, 
            'subsample': 0.8, 
            'leaf_estimation_iterations': 20,
            'reg_lambda': 0.5,
            # 'class_weights': class_weights,
            'early_stopping_rounds': 200 
        }

scores = cv(train_pool,
            params,
            fold_count=5, 
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 1.3325135	test: 1.3325909	best: 1.3325909 (0)	total: 278ms	remaining: 2m 18s
1:	learn: 1.2824303	test: 1.2826277	best: 1.2826277 (1)	total: 546ms	remaining: 2m 16s
2:	learn: 1.2377607	test: 1.2374572	best: 1.2374572 (2)	total: 839ms	remaining: 2m 18s
3:	learn: 1.1941436	test: 1.1936524	best: 1.1936524 (3)	total: 1.26s	remaining: 2m 36s
4:	learn: 1.1439258	test: 1.1440621	best: 1.1440621 (4)	total: 1.49s	remaining: 2m 27s
5:	learn: 1.1015492	test: 1.1016998	best: 1.1016998 (5)	total: 1.68s	remaining: 2m 18s
6:	learn: 1.0640023	test: 1.0638274	best: 1.0638274 (6)	total: 2.02s	remaining: 2m 22s
7:	learn: 1.0283842	test: 1.0280797	best: 1.0280797 (7)	total: 2.26s	remaining: 2m 18s
8:	learn: 0.9987248	test: 0.9982613	best: 0.9982613 (8)	total: 2.47s	remaining: 2m 14s
9:	learn: 0.9640659	test: 0.9641523	best: 0.9641523 (9)	total: 2.7s	remaining: 2m 12s
10:	learn: 0.9352441	test: 0.9353074	best: 0.9353074 (10)	total: 2.96s	remaining: 2m 11s
11:	learn: 0.902942

In [ ]:

import plotly.express as px 
from IPython.display import display
#feature importance 
dfimportance = model.get_feature_importance(prettified=True) 
dfimportance = dfimportance.sort_values(by = "Importances").iloc[-20:]
fig_importance = px.bar(dfimportance,x="Importances",y="Feature Id",title="Feature Importance")
 
display(dfimportance)
display(fig_importance)
